# 4. 작사가 인공지능 만들기

## Step 1. 데이터 다운로드

이미 실습(1) 데이터 다듬기에서 Cloud shell에 심볼릭 링크 ~/aiffel/lyricist/data를 생성하셨다면, ~/aiffel/lyricist/data/lyrics에 데이터가 있음.


## Step 2. 데이터 읽어오기

glob 모듈을 사용하면 파일을 읽어오는 작업을 하기가 아주 용이함.
glob 를 활용하여 모든 txt 파일을 읽어온 후, raw_corpus 리스트에 문장 단위로 저장.

In [1]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


## Step 3. 데이터 정제

#### 앞서 배운 테크닉들을 활용해 문장 생성에 적합한 모양새로 데이터를 정제.

(preprocess_sentence() 함수를 활용해 데이터를 정제.)
1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
2. 특수문자 양쪽에 공백을 넣고
3. 여러개의 공백은 하나의 공백으로 바꿉니다
4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
5. 다시 양쪽 공백을 지웁니다
6. 문장 시작에는 \<start>, 끝에는 \<end>를 추가합니다

이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있음.

In [2]:
import re 

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 소문자 변경, 양쪽 공백 제거.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 특수문자 양쪽 공백 처리.
    sentence = re.sub(r'[" "]+', " ", sentence) # 다수의 공백을 하나의 공백으로 처리.
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 영어와 지정한 특수문자 제외 공백 처리.
    sentence = sentence.strip() # 양쪽 공백 제거.
    sentence = '<start> ' + sentence + ' <end>' # 문장 시작에는 <start>, 끝에는 <end>를 추가
    return sentence

print(preprocess_sentence(raw_corpus[0]))# 정제 결과 확인

<start> now i ve heard there was a secret chord <end>


추가로 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거.
(너무 긴 문장은 노래 가사 작사하기에 어울리지 않을 수도 있음.)

In [3]:
corpus = []

for sentence in raw_corpus:
    # 원하지 않는 문장은 건너뜀.
    
    if len(sentence) == 0 or len(sentence.split()) > 15: continue
        # 길이가 0인 빈 문장 또는 토큰이 15개가 넘는 문장 건너뜀.
        
    if sentence[-1] == ']': continue
        # ']'로 끝나는 파트를 나눈 문장 건너뜀.
        
    if sentence[-1] == ')': continue
        # ')'로 끝나는 코러스 문장 건너뜀.
        
#    if sentence[-1] == ":": continue # 가사가 ':'으로 마치는 경우 존재하므로 건너뛰지 않는다    
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    
# 정제된 결과 확인
corpus[:7]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>']

In [4]:
# corpus = []
# for idx, sentence in enumerate(raw_corpus):
#     # 아무것도 들어 있지 않은 문장 제외
#     if len(sentence) == 0 : continue
    
#     preprocessed_sentence = preprocess_sentence(sentence)
#     # 정제후 단어의 개수가 13개를 넘는 문장 제외
#     if not ( preprocessed_sentence.count(' ')+1 <= 15) : continue
#     # 정제후 단어의 개수가 1개인 문장 제외
#     if preprocessed_sentence.count(' ')+1 <= 4 : continue
#     # 정제하고 나니 corpus에 이미 등록되어 있는 경우는 건너뜀
#     # if preprocessed_sentence in corpus : continue
#     corpus.append(preprocessed_sentence)

#### 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외.
토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용.


In [5]:
import tensorflow as tf

def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성.
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환.
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춤.
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춤.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   49    5 ...    0    0    0]
 [   2   17 2618 ...    0    0    0]
 [   2   32    7 ...   45    3    0]
 ...
 [   2  258  192 ...   12    3    0]
 [   5   23    9 ...   10 1057    3]
 [   2    7   33 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f7504562b20>


In [6]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx]) # 단어사전 내용 확인

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


 ## Step 4. 평가 데이터셋 분리

### 훈련 데이터와 평가 데이터를 분리.
tokenize() 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리.
#### 단어장의 크기는 12,000 이상 으로 설정.
#### 총 데이터의 20% 를 평가 데이터셋으로 사용.

In [7]:
from sklearn.model_selection import train_test_split

src_input = tensor[:, :-1]  
tgt_input = tensor[:, 1:]    

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size = 0.2)
enc_val_train, enc_val_val, dec_val_train, dec_val_val = train_test_split(enc_train, dec_train, test_size = 0.2) 
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (127837, 14)
Target Train: (127837, 14)


In [8]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 12,000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12,001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듬.

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [9]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (127837, 14)
Target Train: (127837, 14)


## Step 5. 인공지능 만들기

모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계. (Loss는 아래 제시된 Loss 함수를 그대로 사용.)

In [10]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
    
embedding_size = 256
hidden_size = 1024
lyricist = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [11]:
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봄.
lyricist(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-1.44154052e-04, -6.53484822e-05, -1.58454743e-04, ...,
          1.09955217e-04, -7.10229233e-06, -8.26604737e-05],
        [-1.50740190e-04, -3.41225059e-05, -2.73217360e-04, ...,
          3.56156146e-04,  1.07076645e-04, -1.49429587e-04],
        [-2.75510276e-04, -1.41341123e-04, -3.72998882e-04, ...,
          7.20555137e-04,  3.91176181e-05, -3.21971806e-04],
        ...,
        [-1.19180628e-03, -8.89089715e-04, -3.93882510e-04, ...,
          9.87562016e-05,  1.03227410e-03, -1.78727636e-03],
        [-1.03560125e-03, -1.08568359e-03, -7.95994943e-04, ...,
          3.91249050e-05,  1.03292195e-03, -1.70092320e-03],
        [-8.70454474e-04, -1.23841001e-03, -1.16272550e-03, ...,
         -1.09324164e-05,  1.02774799e-03, -1.51110464e-03]],

       [[-1.44154052e-04, -6.53484822e-05, -1.58454743e-04, ...,
          1.09955217e-04, -7.10229233e-06, -8.26604737e-05],
        [-2.90683674e-04,  2.30848047e-04, -3

In [12]:
lyricist.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [13]:
optimizer = tf.keras.optimizers.Adam()
# optimizer = tf.keras.optimizers.SGD()

#Loss
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

lyricist.compile(loss=loss, optimizer=optimizer)
lyricist.fit(dataset, epochs=10)

Epoch 1/10
624/624 [==============================] - 98s 153ms/step - loss: 3.6408
Epoch 2/10
624/624 [==============================] - 101s 161ms/step - loss: 3.1224
Epoch 3/10
624/624 [==============================] - 103s 164ms/step - loss: 2.9354
Epoch 4/10
624/624 [==============================] - 104s 166ms/step - loss: 2.7850
Epoch 5/10
624/624 [==============================] - 104s 166ms/step - loss: 2.6545
Epoch 6/10
624/624 [==============================] - 104s 167ms/step - loss: 2.5323
Epoch 7/10
624/624 [==============================] - 104s 166ms/step - loss: 2.4165
Epoch 8/10
624/624 [==============================] - 104s 166ms/step - loss: 2.3073
Epoch 9/10
624/624 [==============================] - 104s 166ms/step - loss: 2.2045
Epoch 10/10
624/624 [==============================] - 104s 167ms/step - loss: 2.1064


In [14]:
def generate_text(lyricist, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = lyricist(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

모델이 생성한 가사 한 줄을 제출.

In [15]:
generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i m better <end> '